### Use this notebook for developing mlframe prototype 2

In [1]:
from commands import commands
commands.lscom()

Scripts location: /Users/peterbryzgalov/work/ML/mlframework/mlframe/commands/../scripts
execremote 		Execute command on remote machine
test_nvidia_ubuntu 		Get versions of installed Nvidia drivers and CUDA
test_docker_ubuntu 		Get versions of Docker and nvidia-docker
install_docker_ubuntu 		Install Docker on Ubuntu
install_nvdocker_ubuntu 		Install nvidia-docker on Ubuntu
test 		Test script with sleep


In [2]:
commands.remote("mouse","test")

Command /Users/peterbryzgalov/work/ML/mlframework/mlframe/commands/../scripts/test.sh


In [ ]:
commands.execremote("mouse","test.sh")

In [ ]:
commands.execremote("mouse","test_nvidia_ubuntu")

In [ ]:
commands.test_nvidia()